<center>
<img src="../../img/ods_stickers.jpg">
## Open Machine Learning Course
<center>Author: [Yury Kashnitsky](https://www.linkedin.com/in/festline/), Data Scientist @ Mail.Ru Group <br>All content is distributed under the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license.

# <center> Assignment #10 (demo)
## <center> Gradient boosting

Your task is to beat at least 2 benchmarks in this [Kaggle Inclass competition](https://www.kaggle.com/c/flight-delays-spring-2018). Here you won’t be provided with detailed instructions. We only give you a brief description of how the second benchmark was achieved using Xgboost. Hopefully, at this stage of the course, it's enough for you to take a quick look at the data in order to understand that this is the type of task where gradient boosting will perform well. Most likely it will be Xgboost, however, we’ve got plenty of categorical features here.

<img src='../../img/xgboost_meme.jpg' width=40% />

In [53]:
import warnings

warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

In [54]:
train = pd.read_csv("../../data/flight_delays_train.csv")
test = pd.read_csv("../../data/flight_delays_test.csv")

In [55]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [56]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


Given flight departure time, carrier's code, departure airport, destination location, and flight distance, you have to predict departure delay for more than 15 minutes. As the simplest benchmark, let's take Xgboost classifier and two features that are easiest to take: DepTime and Distance. Such model results in 0.68202 on the LB.

In [57]:
X_train = train[["Distance", "DepTime"]].values
y_train = train["dep_delayed_15min"].map({"Y": 1, "N": 0}).values
X_test = test[["Distance", "DepTime"]].values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(
    X_train, y_train, test_size=0.3, random_state=17
)

We'll train Xgboost with default parameters on part of data and estimate holdout ROC AUC.

In [58]:
xgb_model = XGBClassifier(seed=17)

xgb_model.fit(X_train_part, y_train_part)
xgb_valid_pred = xgb_model.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, xgb_valid_pred)

[12:54:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.7001228548770644

Now we do the same with the whole training set, make predictions to test set and form a submission file. This is how you beat the first benchmark. 

In [59]:
xgb_model.fit(X_train, y_train)
xgb_test_pred = xgb_model.predict_proba(X_test)[:, 1]

pd.Series(xgb_test_pred, name="dep_delayed_15min").to_csv(
    "xgb_2feat.csv", index_label="id", header=True
)

[12:54:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


***So, the goal of the assignment is to surpass benchmark2, which (0.72045 ROC AUC) and benchmark3 (0.72988 ROC AUC)***.

In [115]:
train1 =  train.copy()
test1 = test.copy()

***Advices below recommend to create new feature and drop previous two.***

In [116]:
train1['Flight'] = train1['Origin'] + '-' + train1['Dest']
test1['Flight'] = test1['Origin'] + '-' + test1['Dest']

In [117]:
train1.drop(['Origin', 'Dest'], axis = 1, inplace = True)
test1.drop(['Origin', 'Dest'], axis = 1, inplace = True)

***Also, I suppose, that all features with date are numerical, not categorical, thus removing extra c-.***

In [118]:
train1['Month'] = train1['Month'].map(lambda x: x.lstrip('c-'))
train1['DayofMonth'] = train1['DayofMonth'].map(lambda x: x.lstrip('c-'))
train1['DayOfWeek'] = train1['DayOfWeek'].map(lambda x: x.lstrip('c-'))

test1['Month'] = test1['Month'].map(lambda x: x.lstrip('c-'))
test1['DayofMonth'] = test1['DayofMonth'].map(lambda x: x.lstrip('c-'))
test1['DayOfWeek'] = test1['DayOfWeek'].map(lambda x: x.lstrip('c-'))




In [119]:
train1.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Distance,dep_delayed_15min,Flight
0,8,21,7,1934,AA,732,N,ATL-DFW
1,4,20,3,1548,US,834,N,PIT-MCO
2,9,2,5,1422,XE,416,N,RDU-CLE
3,11,25,6,1015,OO,872,N,DEN-MEM
4,10,7,6,1828,WN,423,Y,MDW-OMA


In [120]:
train1.dtypes

Month                object
DayofMonth           object
DayOfWeek            object
DepTime               int64
UniqueCarrier        object
Distance              int64
dep_delayed_15min    object
Flight               object
dtype: object

In [121]:
train1 = train1.astype({"Month": int, "DayofMonth": int, "DayOfWeek": int})
test1 = test1.astype({"Month": int, "DayofMonth": int, "DayOfWeek": int})

In [122]:
train1.dtypes

Month                 int32
DayofMonth            int32
DayOfWeek             int32
DepTime               int64
UniqueCarrier        object
Distance              int64
dep_delayed_15min    object
Flight               object
dtype: object

***I'll start with dummy variables.***

In [123]:
for i in ['UniqueCarrier', 'Flight']:
    train1 = pd.get_dummies(train1, prefix=[i], columns = [i], drop_first=True)
    test1 = pd.get_dummies(test1, prefix=[i], columns = [i], drop_first=True)

In [124]:
train1.head()

,Month,DayofMonth,DayOfWeek,DepTime,Distance,dep_delayed_15min,UniqueCarrier_AQ,UniqueCarrier_AS,UniqueCarrier_B6,UniqueCarrier_CO,...,Flight_XNA-IAH,Flight_XNA-LAX,Flight_XNA-LGA,Flight_XNA-ORD,Flight_XNA-SLC,Flight_YAK-CDV,Flight_YAK-JNU,Flight_YUM-IPL,Flight_YUM-LAX,Flight_YUM-PHX
0,8,21,7,1934,732,N,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,20,3,1548,834,N,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9,2,5,1422,416,N,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,11,25,6,1015,872,N,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10,7,6,1828,423,Y,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


***Difference in features after using dummy variables should be eliminated.***

In [125]:
print(train1.shape)
print(test1.shape)

(100000, 4455)
(100000, 4692)


In [126]:
y_train = train1["dep_delayed_15min"].map({"Y": 1, "N": 0}).values
train1.drop(['dep_delayed_15min'], axis = 1, inplace = True)

In [127]:
X_train = train1.copy()
X_test = test1.copy()

In [128]:
print(X_train.shape)
print(X_test.shape)

(100000, 4454)
(100000, 4692)


In [129]:
y_valid.shape

(30000,)

***Taking columns that appears only in both of the sets by XORing, then separating by only in X_train/X_train, and finally,  dropping them. It is much more faster way, than going by loop and checking if it is in the other frame and drop it.***

In [130]:
features_only_in_train = (X_train.columns ^ X_test.columns) & X_train.columns
features_only_in_test = (X_train.columns ^ X_test.columns) & X_test.columns

In [131]:
X_train.drop(features_only_in_train, axis = 1, inplace = True)

In [132]:
X_test.drop(features_only_in_test, axis = 1, inplace = True)

In [133]:
print(X_train.shape)
print(X_test.shape)

(100000, 4072)
(100000, 4072)


In [134]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(
    X_train, y_train, test_size=0.3, random_state=17
)

***Now all features are the same, so can do fit some models.***

***Following the simplest and quickest way of dealing with fitting the model via Catboost. With such approach of transforming the data - it is the most reliable way to on the local machine. Logistic regression, as well as XGBoost resulted in freezing, while the grid search was active.***

In [ ]:
from catboost import CatBoostClassifier

In [136]:
for i in [10, 50, 100, 250, 500, 750, 1000, 2000, 3000, 4000, 5000]:
    
    model = CatBoostClassifier(random_state=17, learning_rate=0.1, max_depth=10, n_estimators=i)
    model.fit(X_train_part, y_train_part, verbose=False)

    cat_test_pred = model.predict_proba(X_valid)[:, 1]
    
    print(roc_auc_score(y_valid, cat_test_pred))

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0.6954330214599793
0.7153958490915728
0.7208037196306121
0.7261560909422773
0.7273129027361714
0.7278520525955101
0.7272580418478392
0.7265878221231992
0.7227262263590138
0.7202461413583673
0.7175444874255369


***So, the best ROC AUC with such approach is 0.72785, and it is achived with 750 trees.*** 

In [137]:
scaler = StandardScaler()

X_train_part = scaler.fit_transform(X_train_part)
X_valid = scaler.fit_transform(X_valid)

In [138]:
for i in [10, 50, 100, 250, 500, 750]:
    
    model = CatBoostClassifier(random_state=17, learning_rate=0.1, max_depth=10, n_estimators=i)
    model.fit(X_train_part, y_train_part, verbose=False)

    cat_test_pred = model.predict_proba(X_valid)[:, 1]
    
    print(roc_auc_score(y_valid, cat_test_pred))

0.6953746351992116
0.7153151354720473
0.7206837554138676
0.7259393530760234
0.7269373780332666
0.7272747764861286


In [141]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [142]:
model = CatBoostClassifier(random_state=17, learning_rate=0.1, max_depth=10, n_estimators=750)
model.fit(X_train, y_train, verbose=False)

cat_test_pred = model.predict_proba(X_test)[:, 1]

pd.Series(cat_test_pred, name="dep_delayed_15min").to_csv(
    "cat_lim.csv", index_label="id", header=True
)



***Both of the results with and without scaling with a catboost gave a result below 2nd benchmark - 0.71840 and 0.71893 respectfully.***

***Another approach is not to go with dummy variables, but try a label encoding.***

In [193]:
train2 =  train.copy()
test2 = test.copy()

In [194]:
train2['Month'] = train2['Month'].map(lambda x: x.lstrip('c-'))
train2['DayofMonth'] = train2['DayofMonth'].map(lambda x: x.lstrip('c-'))
train2['DayOfWeek'] = train2['DayOfWeek'].map(lambda x: x.lstrip('c-'))

test2['Month'] = test2['Month'].map(lambda x: x.lstrip('c-'))
test2['DayofMonth'] = test2['DayofMonth'].map(lambda x: x.lstrip('c-'))
test2['DayOfWeek'] = test2['DayOfWeek'].map(lambda x: x.lstrip('c-'))

In [195]:
train2 = train2.astype({"Month": int, "DayofMonth": int, "DayOfWeek": int})
test2 = test2.astype({"Month": int, "DayofMonth": int, "DayOfWeek": int})

In [198]:
carr_dict = {}
UniqueCarrier = train2["UniqueCarrier"].unique()


In [200]:
num = 1
for i in UniqueCarrier:
    carr_dict[i] = num
    num += 1

In [208]:
dest_dict = {}
dest = train2["Dest"].unique()

In [209]:
num = 1
for i in dest:
    dest_dict[i] = num
    num += 1

In [211]:
train2['Dest'] = train2['Dest'].map(dest_dict)


In [213]:
train2['Origin'] = train2['Origin'].map(dest_dict)
train2['UniqueCarrier'] = train2['UniqueCarrier'].map(carr_dict)

In [214]:
test2['Dest'] = test2['Dest'].map(dest_dict)
test2['Origin'] = test2['Origin'].map(dest_dict)
test2['UniqueCarrier'] = test2['UniqueCarrier'].map(carr_dict)

In [217]:
train2.drop(['dep_delayed_15min'], axis = 1, inplace = True)

In [218]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(
    train2, y_train, test_size=0.3, random_state=17
)

In [219]:
for i in [10, 50, 100, 250, 500, 750, 1000, 2000]:
    
    model = CatBoostClassifier(random_state=17, learning_rate=0.1, max_depth=10, n_estimators=i)
    model.fit(X_train_part, y_train_part, verbose=False)

    cat_test_pred = model.predict_proba(X_valid)[:, 1]
    
    print(roc_auc_score(y_valid, cat_test_pred))

0.6927985635920788
0.7132375667603804
0.7222161934646052
0.7319716026172626
0.7290009830783937
0.7256043157522308
0.7224978968360689
0.7155336759207088


In [220]:
model = CatBoostClassifier(random_state=17, learning_rate=0.1, max_depth=10, n_estimators=250)
model.fit(train2, y_train, verbose=False)

cat_test_pred = model.predict_proba(test2)[:, 1]

pd.Series(cat_test_pred, name="dep_delayed_15min").to_csv(
    "cat_lim.csv", index_label="id", header=True
)

***The result from Kaggle competition is 0.72221, which is surpassing 2nd benchmark, thus the assignment is completed.***

In [221]:
for i in [10, 50, 100, 250, 500, 750, 1000]:
    for j in [0.1, 0.03, 0.01]:
        for k in range (4, 8, 2):
            print(i, j, k)
            model = CatBoostClassifier(random_state=17, learning_rate = j, max_depth = k, n_estimators=i)
            model.fit(X_train_part, y_train_part, verbose=False)

            cat_test_pred = model.predict_proba(X_valid)[:, 1]
        
            print(roc_auc_score(y_valid, cat_test_pred))

10 0.1 4
0.6870971695285232
10 0.1 6
0.689041887554039
10 0.03 4
0.6837963038356016
10 0.03 6
0.6844952887133133
10 0.01 4
0.6821217736903122
10 0.01 6
0.6844352050510027
50 0.1 4
0.6997476130744168
50 0.1 6
0.7052367376450113
50 0.03 4
0.6903409111310552
50 0.03 6
0.6923693966481035
50 0.01 4
0.6826464521134191
50 0.01 6
0.6864084343447441
100 0.1 4
0.7090754583845561
100 0.1 6
0.7160255088985732
100 0.03 4
0.6959778111898296
100 0.03 6
0.6996806164904372
100 0.01 4
0.6862799178356099
100 0.01 6
0.6892834408505432
250 0.1 4
0.7272831909553161
250 0.1 6
0.7339675799932213
250 0.03 4
0.7082352663823277
250 0.03 6
0.7149079658365298
250 0.01 4
0.6966892400519303
250 0.01 6
0.7004361850465091
500 0.1 4
0.73541917748201
500 0.1 6
0.7400401283134012
500 0.03 4
0.7184342595112971
500 0.03 6
0.7275506259984248
500 0.01 4
0.7033620410600708
500 0.01 6
0.7091575865053676
750 0.1 4
0.7381889523459958
750 0.1 6
0.7414440317203727
750 0.03 4
0.7254720852704898
750 0.03 6
0.7337530835674666
750 0.0

***Using the model with best ROC AUC here leads to 0.72880 in Kaggle competition, which is even better than previous and closing to the 3rd benchmark.***

In [222]:
model = CatBoostClassifier(random_state=17, learning_rate=0.1, max_depth=6, n_estimators=1000)
model.fit(train2, y_train, verbose=False)

cat_test_pred = model.predict_proba(test2)[:, 1]

pd.Series(cat_test_pred, name="dep_delayed_15min").to_csv(
    "cat_lim.csv", index_label="id", header=True
)

In [224]:
for i in [10, 50, 100, 250, 500, 750, 1000]:
    for j in [0.1, 0.03, 0.01]:
        for k in range (3, 10, 2):
            
            model = CatBoostClassifier(random_state=17, learning_rate = j, max_depth = k, n_estimators=i)
            model.fit(X_train_part, y_train_part, verbose=False)

            cat_test_pred = model.predict_proba(X_valid)[:, 1]
            roc_auc = roc_auc_score(y_valid, cat_test_pred)
            if(roc_auc > 0.72):
                print(i, j, k)
                print(roc_auc_score(y_valid, cat_test_pred))

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

100 0.1 9
0.7216518872439367
250 0.1 3
0.7206417338448992
250 0.1 5
0.7314585158508471
250 0.1 7
0.7351797205489499
250 0.1 9
0.7343747387479596
250 0.03 9
0.7223629679211055
500 0.1 3
0.7298193047145871
500 0.1 5
0.7379884195867806
500 0.1 7
0.7403956541442802
500 0.1 9
0.7337887797768389
500 0.03 5
0.7230583730224863
500 0.03 7
0.7294529271199974
500 0.03 9
0.7323349335936983
750 0.1 3
0.7334615657332348
750 0.1 5
0.7402983219481092
750 0.1 7
0.7406746373209636
750 0.1 9
0.731062165336703
750 0.03 5
0.729839883894833
750 0.03 7
0.7354321546245759
750 0.03 9
0.736164003082162
750 0.01 9
0.7230790537566705
1000 0.1 3
0.7359172922963539
1000 0.1 5
0.740800237781293
1000 0.1 7
0.740810712344661
1000 0.1 9
0.7298111151148348
1000 0.03 3
0.7229400336687575
1000 0.03 5
0.7336648839719051
1000 0.03 7
0.7382416660939146
1000 0.03 9
0.7372296956047637
1000 0.01 7
0.7224967579811875
1000 0.01 9
0.7271106145445914


In [226]:
model = CatBoostClassifier(random_state=17, learning_rate=0.1, max_depth=5, n_estimators=1000)
model.fit(train2, y_train, verbose=False)

cat_test_pred = model.predict_proba(test2)[:, 1]

pd.Series(cat_test_pred, name="dep_delayed_15min").to_csv(
    "cat_lim.csv", index_label="id", header=True
)

***After some more tuning above, the model with parameters learning_rate=0.1, max_depth=5, n_estimators=1000 gave Kaggle ROC AUC on the testing dataset 0.73027, which is beating all 3 set benchmarks for the competition.***

***Thus, the resulted model (ROC AUC from Kaggle after submission is 0.73027) is better than:***
- benchmark1, ROC AUC 0.68202;
- benchmark2, ROC AUC 0.72045;
- benchmark3, ROC AUC 0.72988;
- 65% of other submissions.

***As the goal was to beat 2 out of 3 benchmarks, and all 3 are surpassed, there is no more reson to try to tune the model, or look of another approach. The resulted model is approximately in top 35% of all submissions.***


***XGBoost.***

In [234]:
from xgboost.sklearn import XGBClassifier

In [244]:
for i in [10, 25, 50, 75, 100, 150, 200, 250, 500, 750, 1000, 2000]:
    for j in [0.1, 0.03, 0.01]:
        mod = XGBClassifier(learning_rate = j, n_estimators=i, seed=17, eval_metric='mlogloss');
        mod.fit(X_train_part, y_train_part);
        xgb_valid_pred = mod.predict(X_valid);
    
        roc_auc = roc_auc_score(y_valid, xgb_valid_pred)
    
        print(roc_auc, i, j)

0.5289984913799748 10 0.1
0.5268125065851382 10 0.03
0.5258682544383152 10 0.01
0.5289438154648876 25 0.1
0.5271678728312567 25 0.03
0.527145005060469 25 0.01
0.5329954367825827 50 0.1
0.5275704072548683 50 0.03
0.524672257331877 50 0.01
0.5386189679907426 75 0.1
0.5277964191718603 75 0.03
0.5269114854058831 75 0.01
0.5423418447019038 100 0.1
0.5288700256478098 100 0.03
0.5283722662722505 100 0.01
0.5494840425213594 150 0.1
0.5313534909438369 150 0.03
0.527644197071946 150 0.01
0.5536108862920499 200 0.1
0.5346579291592368 200 0.03
0.5273695843415432 200 0.01
0.5569228322450439 250 0.1
0.537479968032271 250 0.03
0.5279813380130355 250 0.01
0.5660087543121888 500 0.1
0.5479957514184456 500 0.03
0.5325153586730962 500 0.01
0.5718466912729442 750 0.1
0.554329126521881 750 0.03
0.5373856316772847 750 0.01
0.5751782951668848 1000 0.1
0.5584148772167548 1000 0.03
0.5414676285033615 1000 0.01
0.5829483502109329 2000 0.1
0.5679953488296181 2000 0.03
0.5503003729749737 2000 0.01


***The rest of the approaches, that can be tried are either are computationally inefficient on the local device or perform much worse than Catboost approach.***

The second benchmark in the leaderboard was achieved as follows:

- Features `Distance` and `DepTime` were taken unchanged
- A feature `Flight` was created from features `Origin` and `Dest`
- Features `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` and `Flight` were transformed with OHE (`LabelBinarizer`)
- Logistic regression and gradient boosting (xgboost) were trained. Xgboost hyperparameters were tuned via cross-validation. First, the hyperparameters responsible for model complexity were optimized, then the number of trees was fixed at 500 and learning step was tuned.
- Predicted probabilities were made via cross-validation using `cross_val_predict`. A linear mixture of logistic regression and gradient boosting predictions was set in the form $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, where $p_{logit}$ is a probability of class 1, predicted by logistic regression, and $p_{xgb}$ – the same for xgboost. $w_1$ weight was selected manually.
- A similar combination of predictions was made for test set. 

Following the same steps is not mandatory. That’s just a description of how the result was achieved by the author of this assignment. Perhaps you might not want to follow the same steps, and instead, let’s say, add a couple of good features and train a random forest of a thousand trees.

Good luck!